1.提取文本特征


In [10]:
from text import chinese
import os 
import numpy as np
finals= ['a','ai','an','ang','ao','e','ei','en','eng','er','i','i0','ia','ian','iang','iao','ie','in','ing','iong','ir','iu','o','ong','ou','u','ua','uai','uan','uang','ui','un','uo','v','van','ve','vn']

def generate_token(text):
    
    phone_set = "./text/chinese_dict"
    with open(phone_set) as ps:
        phone_map = { pho.strip():idx+1 for idx,pho in enumerate(ps.readlines())}

    norm_text = chinese.text_normalize(text)
    phones, tones, word2ph = chinese.g2p(norm_text)

    final_phone_lst = []
    for phone,tone in zip(phones,tones):
        if '_' == phone:
            continue
        if phone in finals:
            phone = phone+str(tone)
        if phone == '!' or phone == '…':
            phone = '.'
        if phone == '-':
            phone = ','
        final_phone_lst.append(phone.strip())
    if 'iong4' in final_phone_lst:                              # 默认音素集内不存在iong4
        assert 0
    token_lst = [phone_map[phone] for phone in final_phone_lst]

    return np.array(token_lst)
    

In [11]:
prompt_text = '然后在家看书的时候，就发现那些什么心灵鸡汤啊、心灵鸡血呀，还有什么毒鸡汤啊，还挺管用的。'    #   提示文本
target_text = '你好，我是郭钊。'                                                                #   待合成文本

In [12]:
token = generate_token(prompt_text)
np.save("prompt_text_token.npy",token)
token = generate_token(target_text)
np.save("target_text_token.npy",token)

2.以下代码需要在Amphion根目录下运行

In [1]:
import argparse
import os
import torch
import torchaudio
import numpy as np
from utils.io import save_audio
from tqdm import tqdm
from utils.tokenizer import AudioTokenizer,extract_encodec_token
from utils.util import load_config
from models.tts.valle.valle import VALLE
from encodec import EncodecModel
from encodec.utils import convert_audio

os.environ['WORK_DIR'] = '.'
os.environ['PYTHONPATH'] = '.'
os.environ['PYTHONIOENCODING'] = 'UTF-8'

/home/environment/kxxia/anaconda3/envs/amphion/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
in_config = './egs/tts/VALLE/exp_config.json'
in_ckpt_path = '/home/work_nfs14/kxxia/work/Amphion/Amphion/ckpts/tts/valle/wenet_finetune_together_4/checkpoint/epoch-0047_step-0029160_loss-2.991779/pytorch_model.bin'


cfg = load_config(in_config)
model = VALLE(cfg.model)
audio_tokenizer = AudioTokenizer()
ckpt = torch.load(in_ckpt_path)
model.load_state_dict(ckpt)

enc_model = EncodecModel.encodec_model_24khz()
enc_model.set_target_bandwidth(6.0)
enc_model = enc_model.cuda()
enc_model.eval()

EncodecModel(
  (encoder): SEANetEncoder(
    (model): Sequential(
      (0): SConv1d(
        (conv): NormConv1d(
          (conv): Conv1d(1, 32, kernel_size=(7,), stride=(1,))
          (norm): Identity()
        )
      )
      (1): SEANetResnetBlock(
        (block): Sequential(
          (0): ELU(alpha=1.0)
          (1): SConv1d(
            (conv): NormConv1d(
              (conv): Conv1d(32, 16, kernel_size=(3,), stride=(1,))
              (norm): Identity()
            )
          )
          (2): ELU(alpha=1.0)
          (3): SConv1d(
            (conv): NormConv1d(
              (conv): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
              (norm): Identity()
            )
          )
        )
        (shortcut): SConv1d(
          (conv): NormConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
            (norm): Identity()
          )
        )
      )
      (2): ELU(alpha=1.0)
      (3): SConv1d(
        (conv): NormConv1d(
          (conv): Co

In [10]:
prompt_audio_path = '/home/work_nfs14/lhma/wenetTTS/wenetTTS_testset/ref_timbre/wavs/TEST_NET_Y0000000076_a7UyOEAf4vE_S00023.wav'       #提示音频路径
prompt_text_token_path = '/home/work_nfs14/kxxia/work/ISCSLP2024_CoVoC_baseline/prompt_text_token.npy'                                  #1中提取的提示文本特征路径
target_text_token_path = '/home/work_nfs14/kxxia/work/ISCSLP2024_CoVoC_baseline/target_text_token.npy'                                  #1中提取的待合成文本特征路径

In [11]:
wav, sr = torchaudio.load(prompt_audio_path)
wav = convert_audio(wav, sr, enc_model.sample_rate, enc_model.channels)
wav = wav.unsqueeze(0)
wav = wav.cuda()

with torch.no_grad():
    encoded_frames = enc_model.encode(wav)
    codes_ = torch.cat(
        [encoded[0] for encoded in encoded_frames], dim=-1
    )  # [B, n_q, T]
    codes = codes_.cpu().numpy()[0, :, :].T  # [T, 8]

prompt_audio_token = codes                          # encodec 特征

In [12]:
prompt_text_token = np.load(prompt_text_token_path)
target_text_token = np.load(target_text_token_path)

In [13]:
all_phone =  np.concatenate((prompt_text_token,target_text_token))# prompt_text + prompt_text
semantic_token = torch.from_numpy(all_phone)

semantic_len = semantic_token.shape[0]
semantic_len = torch.IntTensor([semantic_len])

prompt_token = torch.from_numpy(prompt_audio_token)

device = 'cuda:0'
model = model.to(device)
semantic_token = semantic_token.to(device)
semantic_len = semantic_len.to(device)
prompt_token = prompt_token.to(device)
# prompt_phones = torch.tensor(prompt_phones).to(device)
encoded_frames = model.inference(
                semantic_token.unsqueeze(0),
                semantic_len,
                prompt_token.unsqueeze(0),
                enroll_x_lens=len(prompt_text_token),
                top_k=100,
                temperature=1.0,
            )
# break
samples = audio_tokenizer.decode([(encoded_frames.transpose(2, 1), None)])
audio = samples[0].squeeze(0).cpu().detach()

In [14]:
save_audio('target.wav', audio, 24000)